## Load Dataset from HF Hub

---



In [38]:
model_checkpoint = "nvidia/mit-b5"  # pre-trained model from which to fine-tune
batch_size = 3  # batch size for training and evaluation

In [39]:
# Install dependencies
!pip install -q transformers datasets evaluate
!git lfs install
!git config --global credential.helper store

Error: Failed to call git rev-parse --git-dir: exit status 128 
Git LFS initialized.


In [40]:
# Log in to huggingface hub
from huggingface_hub import login

access_token = "" # HF USER ACCESS TOKEN
login(access_token) 
hf_username = "" # HF USER NAME

Token will not been saved to git credential helper. Pass `add_to_git_credential=True` if you want to set the git credential as well.
Token is valid.
Your token has been saved to /root/.cache/huggingface/token
Login successful


In [41]:
# Load dataset from HF
from datasets import load_dataset

hf_dataset_identifier = "" # ENTER HF DATASET ID

ds = load_dataset(hf_dataset_identifier)

  0%|          | 0/1 [00:00<?, ?it/s]

In [42]:
import evaluate

metric = evaluate.load("mean_iou")

In [43]:
from huggingface_hub import hf_hub_download
import json

filename = "id2label.json"
id2label = json.load(
    open(hf_hub_download(hf_dataset_identifier, filename, repo_type="dataset"), "r")
)
id2label = {int(k): v for k, v in id2label.items()}
label2id = {v: k for k, v in id2label.items()}

num_labels = len(id2label)

In [44]:
# Split into train and test
ds = ds.shuffle()
ds = ds["train"].train_test_split(test_size=0.1)
train_ds = ds["train"]
test_ds = ds["test"]

In [45]:
from transformers import AutoFeatureExtractor

feature_extractor = AutoFeatureExtractor.from_pretrained(model_checkpoint)

loading configuration file preprocessor_config.json from cache at /root/.cache/huggingface/hub/models--nvidia--mit-b5/snapshots/40357155205b036cf11b61f132d53d2f8861f170/preprocessor_config.json
/usr/local/lib/python3.8/dist-packages/transformers/models/segformer/feature_extraction_segformer.py:28: FutureWarning: The class SegformerFeatureExtractor is deprecated and will be removed in version 5 of Transformers. Please use SegformerImageProcessor instead.
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/transformers/models/segformer/image_processing_segformer.py:102: FutureWarning: The `reduce_labels` parameter is deprecated and will be removed in a future version. Please use `do_reduce_labels` instead.
  warnings.warn(
size should be a dictionary on of the following set of keys: ({'height', 'width'}, {'shortest_edge'}, {'longest_edge', 'shortest_edge'}), got 512. Converted to {'height': 512, 'width': 512}.
Image processor SegformerFeatureExtractor {
  "do_normalize": true,
  "do_

In [46]:

def train_transforms(example_batch):
    images = [(x) for x in example_batch['pixel_values']]
    labels = [(x) for x in example_batch['label']]
    inputs = feature_extractor(images, labels)
    return inputs


def val_transforms(example_batch):
    images = [x for x in example_batch['pixel_values']]
    labels = [x for x in example_batch['label']]
    inputs = feature_extractor(images, labels)
    return inputs


# Set transforms
train_ds.set_transform(train_transforms)
test_ds.set_transform(val_transforms)

In [47]:
from transformers import SegformerForSemanticSegmentation

model = SegformerForSemanticSegmentation.from_pretrained(
    model_checkpoint,
    num_labels=num_labels,
    id2label=id2label,
    label2id=label2id,
    ignore_mismatched_sizes=True,  # Will ensure the segmentation specific components are reinitialized.
)

loading configuration file config.json from cache at /root/.cache/huggingface/hub/models--nvidia--mit-b5/snapshots/40357155205b036cf11b61f132d53d2f8861f170/config.json
Model config SegformerConfig {
  "architectures": [
    "SegformerForImageClassification"
  ],
  "attention_probs_dropout_prob": 0.0,
  "classifier_dropout_prob": 0.1,
  "decoder_hidden_size": 768,
  "depths": [
    3,
    6,
    40,
    3
  ],
  "downsampling_rates": [
    1,
    4,
    8,
    16
  ],
  "drop_path_rate": 0.1,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.0,
  "hidden_sizes": [
    64,
    128,
    320,
    512
  ],
  "id2label": {
    "0": "unlabeled",
    "1": "trash"
  },
  "image_size": 224,
  "initializer_range": 0.02,
  "label2id": {
    "trash": 1,
    "unlabeled": 0
  },
  "layer_norm_eps": 1e-06,
  "mlp_ratios": [
    4,
    4,
    4,
    4
  ],
  "model_type": "segformer",
  "num_attention_heads": [
    1,
    2,
    5,
    8
  ],
  "num_channels": 3,
  "num_encoder_blocks": 4,
  "patch_si

In [48]:
from transformers import TrainingArguments

epochs = 10
lr = 0.00006
# batch_size = 3

hub_model_id = "trashbot"

training_args = TrainingArguments(
    f"{hub_model_id}-outputs16",
    learning_rate=lr,
    num_train_epochs=epochs,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    save_total_limit=3,
    evaluation_strategy="epoch",
    # evaluation_strategy="steps",
    save_strategy="epoch",
    # save_strategy="steps",
    save_steps=20,
    eval_steps=20,
    logging_steps=1,
    eval_accumulation_steps=5,
    load_best_model_at_end=True,
    push_to_hub=True,
    push_to_hub_model_id=hub_model_id,
    hub_model_id=hub_model_id,
    hub_strategy="end",
)

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).
/usr/local/lib/python3.8/dist-packages/transformers/training_args.py:1334: FutureWarning: `--push_to_hub_model_id` is deprecated and will be removed in version 5 of 🤗 Transformers. Use `--hub_model_id` instead and pass the full repo name to this argument (in this case mraottth/trashbot).
  warnings.warn(


In [49]:
import torch
from torch import nn
import evaluate

metric = evaluate.load("mean_iou")

def compute_metrics(eval_pred):
  with torch.no_grad():
    logits, labels = eval_pred
    logits_tensor = torch.from_numpy(logits)
    # scale the logits to the size of the label
    logits_tensor = nn.functional.interpolate(
        logits_tensor,
        size=labels.shape[-2:],
        mode="bilinear",
        align_corners=False,
    ).argmax(dim=1)

    pred_labels = logits_tensor.detach().cpu().numpy()

    metrics = metric._compute(
            predictions=pred_labels,
            references=labels,
            num_labels=len(id2label),
            ignore_index=0,            
            reduce_labels=feature_extractor.do_reduce_labels,
        )
    
    # add per category metrics as individual key-value pairs
    per_category_accuracy = metrics.pop("per_category_accuracy").tolist()
    per_category_iou = metrics.pop("per_category_iou").tolist()

    metrics.update({f"accuracy_{id2label[i]}": v for i, v in enumerate(per_category_accuracy)})
    metrics.update({f"iou_{id2label[i]}": v for i, v in enumerate(per_category_iou)})
    
    return metrics

In [50]:
from transformers import Trainer

trainer = Trainer(
    model=model,
    args=training_args,
    # tokenizer=feature_extractor,
    train_dataset=train_ds,
    eval_dataset=test_ds,
    compute_metrics=compute_metrics,
)

Cloning https://huggingface.co/mraottth/trashbot into local empty directory.


In [51]:
trainer.train()

/usr/local/lib/python3.8/dist-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 270
  Num Epochs = 10
  Instantaneous batch size per device = 3
  Total train batch size (w. parallel, distributed & accumulation) = 3
  Gradient Accumulation steps = 1
  Total optimization steps = 900
  Number of trainable parameters = 84594882


Epoch,Training Loss,Validation Loss,Mean Iou,Mean Accuracy,Overall Accuracy,Accuracy Unlabeled,Accuracy Trash,Iou Unlabeled,Iou Trash
1,0.059200,0.038698,0.372324,0.744648,0.744648,nan,0.744648,0.000000,0.744648
2,0.040200,0.028111,0.412326,0.824653,0.824653,nan,0.824653,0.000000,0.824653
3,0.020900,0.024630,0.369109,0.738218,0.738218,nan,0.738218,0.000000,0.738218
4,0.011700,0.020975,0.388155,0.776311,0.776311,nan,0.776311,0.000000,0.776311
5,0.019000,0.019814,0.382212,0.764423,0.764423,nan,0.764423,0.000000,0.764423
6,0.044500,0.019877,0.377084,0.754168,0.754168,nan,0.754168,0.000000,0.754168
7,0.019500,0.019067,0.417713,0.835426,0.835426,nan,0.835426,0.000000,0.835426
8,0.008000,0.019069,0.405954,0.811908,0.811908,nan,0.811908,0.000000,0.811908
9,0.026800,0.018848,0.408286,0.816572,0.816572,nan,0.816572,0.000000,0.816572
10,0.006100,0.018882,0.405036,0.810072,0.810072,nan,0.810072,0.000000,0.810072


***** Running Evaluation *****
  Num examples = 30
  Batch size = 3
/root/.cache/huggingface/modules/evaluate_modules/metrics/evaluate-metric--mean_iou/08bc20f4f895f3caf75fb9e3fada1404bded3c3265243d05327cbb3b9326ffe9/mean_iou.py:260: RuntimeWarning: invalid value encountered in true_divide
  acc = total_area_intersect / total_area_label
Saving model checkpoint to trashbot-outputs16/checkpoint-90
Configuration saved in trashbot-outputs16/checkpoint-90/config.json
Model weights saved in trashbot-outputs16/checkpoint-90/pytorch_model.bin
***** Running Evaluation *****
  Num examples = 30
  Batch size = 3
/root/.cache/huggingface/modules/evaluate_modules/metrics/evaluate-metric--mean_iou/08bc20f4f895f3caf75fb9e3fada1404bded3c3265243d05327cbb3b9326ffe9/mean_iou.py:260: RuntimeWarning: invalid value encountered in true_divide
  acc = total_area_intersect / total_area_label
Saving model checkpoint to trashbot-outputs16/checkpoint-180
Configuration saved in trashbot-outputs16/checkpoint-180/co

TrainOutput(global_step=900, training_loss=0.043446575814288936, metrics={'train_runtime': 4583.0793, 'train_samples_per_second': 0.589, 'train_steps_per_second': 0.196, 'total_flos': 1.0777555803045888e+18, 'train_loss': 0.043446575814288936, 'epoch': 10.0})

In [52]:
kwargs = {
    "tags": ["vision", "image-segmentation"],
    "finetuned_from": "nvidia/mit-b5" ,
    "dataset": hf_dataset_identifier,
}

trainer.push_to_hub(**kwargs)

Saving model checkpoint to trashbot-outputs16
Configuration saved in trashbot-outputs16/config.json
Model weights saved in trashbot-outputs16/pytorch_model.bin


Upload file pytorch_model.bin:   0%|          | 32.0k/323M [00:00<?, ?B/s]

Upload file runs/Jan30_04-39-09_7680168615d6/events.out.tfevents.1675053556.7680168615d6.29096.4:  22%|##1    …

Upload file runs/Jan30_04-39-09_7680168615d6/1675053556.847212/events.out.tfevents.1675053556.7680168615d6.290…

Upload file training_args.bin: 100%|##########| 3.43k/3.43k [00:00<?, ?B/s]

remote: Scanning LFS files for validity, may be slow...        
remote: LFS file scan complete.        
To https://huggingface.co/mraottth/trashbot
   18af9a4..d7d85c7  main -> main

remote: LFS file scan complete.        
To https://huggingface.co/mraottth/trashbot
   18af9a4..d7d85c7  main -> main

Dropping the following result as it does not have all the necessary fields:
{}
To https://huggingface.co/mraottth/trashbot
   d7d85c7..01a8273  main -> main

   d7d85c7..01a8273  main -> main



'https://huggingface.co/mraottth/trashbot/commit/d7d85c7971eabc48da1f65c53bf1b894b0ecc12b'

In [53]:
feature_extractor.push_to_hub(hub_model_id)

Image processor saved in /tmp/tmp1bp7jhg1/preprocessor_config.json
Uploading the following files to mraottth/trashbot: preprocessor_config.json


CommitInfo(commit_url='https://huggingface.co/mraottth/trashbot/commit/efec50c92c690a44d3d92288b632f188eecd4c4c', commit_message='Upload feature extractor', commit_description='', oid='efec50c92c690a44d3d92288b632f188eecd4c4c', pr_url=None, pr_revision=None, pr_num=None)